In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd
import polars as pl
from pathlib import Path

from linearmodels.panel import PanelOLS
from statsmodels.tsa.stattools import adfuller

OUPUT_PATH = Path("../latex/imgs/res/")
OUPUT_PATH.mkdir(parents=True, exist_ok=True)
OUPUT_TABLES_PATH = Path("../latex/tables/")
OUPUT_TABLES_PATH.mkdir(parents=True, exist_ok=True)
DATA_OUTPUT_PATH = Path("../data/results/")
DATA_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

DUOPOLY_OUPUT_PATH = Path(OUPUT_PATH) / "duopoly"
DUOPOLY_OUPUT_PATH.mkdir(parents=True, exist_ok=True)

In [2]:
df = (
    pl.read_parquet(DATA_OUTPUT_PATH / "all_experiments.parquet")
    .filter((pl.col("num_agents") == 4) & (pl.col("is_symmetric")))
    .with_columns(
        (pl.col("experiment_timestamp").rank("dense")).alias("run_id"),
        (pl.col("agent").rank("dense")).alias("firm_id"),
        pl.col("chosen_price").truediv(pl.col("alpha")).round(4).alias("price"),
    )
    .rename(
        {
            "round": "period",
            "price": "price",
            "agent_prefix_type": "prompt_prefix",
        }
    )
    .with_columns(
        # concat run_id and firm_id to create a unique identifier
        pl.concat_str(["run_id", "firm_id"], separator="_").alias("run_firm_id")
    )
    .select(["run_firm_id", "run_id", "firm_id", "period", "price", "prompt_prefix"])
    .sort(["run_id", "firm_id", "period"])
)
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix
str,u32,u32,i64,f64,str
"""1_1""",1,1,1,4.51,"""P1"""
"""1_1""",1,1,2,4.0,"""P1"""
"""1_1""",1,1,3,3.5,"""P1"""
"""1_1""",1,1,4,3.0,"""P1"""
"""1_1""",1,1,5,2.75,"""P1"""
…,…,…,…,…,…
"""42_4""",42,4,296,1.2705,"""P2"""
"""42_4""",42,4,297,1.269,"""P2"""
"""42_4""",42,4,298,1.267,"""P2"""


# Stationarity check
---

- https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html

The null hypothesis of the Augmented Dickey-Fuller is that there is a unit root, with the alternative that there is no unit root. If the pvalue is above a critical size, then we cannot reject that there is a unit root.

The p-values are obtained through regression surface approximation from MacKinnon 1994, but using the updated 2010 tables. If the p-value is close to significant, then the critical values should be used to judge whether to reject the null.

The autolag option and maxlag for it are described in Greene.

In [3]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .select("price")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False                65
True                103

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0          26_2       -1.734914  4.132195e-01       False
1          34_4       -0.650129  8.592481e-01       False
2           1_2       -0.994473  7.552747e-01       False
3          26_4       -7.189057  2.531660e-10        True
4          19_4        1.208067  9.960411e-01       False
..          ...             ...           ...         ...
163        38_4       -2.212858  2.016117e-01       False
164         5_1       -2.072425  2.557461e-01       False
165        17_1      -11.372270  8.919184e-21        True
166         4_3       -2.099287  2.448015e-01       False
167         5_3       -7.055780  5.380089e-10        True

[168 rows x 4 columns]

Number of stationary series: 103
Number of non-stationary series: 65


In [4]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .with_columns(
            #   pl.col("price").diff(1).fill_null(0).alias("price_diff")
            (pl.col("price").log())
            .diff(1)
            .fill_null(0)
            .alias("price_diff")  # NOTE! This uses log differences
        )
        .select("price_diff")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False                 2
True                166

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0          13_4       -2.889914  4.653345e-02        True
1           6_2      -21.783106  0.000000e+00        True
2           2_4      -20.595056  0.000000e+00        True
3          21_2      -11.469054  5.322234e-21        True
4          10_4       -1.836192  3.626470e-01       False
..          ...             ...           ...         ...
163        19_1       -4.796868  5.514112e-05        True
164        13_3       -7.837926  6.044838e-12        True
165        26_2      -19.441406  0.000000e+00        True
166        38_2       -8.198020  7.358976e-13        True
167        25_4       -5.310505  5.207857e-06        True

[168 rows x 4 columns]

Number of stationary series: 166
Number of non-stationary series: 2


We need to work with price differences.

In [5]:
df = df.with_columns(
    (pl.col("price").log())
    .diff(1)
    .over("run_firm_id")
    .fill_null(0)
    .alias("price_log_diff")  # NOTE! This uses log differences
)
df.head()

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,4.51,"""P1""",0.0
"""1_1""",1,1,2,4.0,"""P1""",-0.120003
"""1_1""",1,1,3,3.5,"""P1""",-0.133531
"""1_1""",1,1,4,3.0,"""P1""",-0.154151
"""1_1""",1,1,5,2.75,"""P1""",-0.087011


# Fixed effects regression (trigger strategy)
---

We are interested in the responsiveness of agents to each other since it is a feature of a reward-punishment strategy. We are interested in stickiness since it measures the persistence of such rewards and punishments.

To measure responsiveness and stickiness, we perform a linear regression with the following model:
$$p_{i,r}^t = \alpha_{i,r} + \gamma p_{i,r}^{t-1} + \delta p_{-i,r}^{t-1}+\epsilon_{i,r}^{t}$$

$$\Delta \log(p_{i,r}^t) =   \gamma \Delta \log(p_{i,r}^{t-1}) + \delta \Delta \log(p_{-i,r}^{t-1})+ \Delta \epsilon_{i,r}^{t}$$


where $p_{i,r}^t$ is the price set by the agent $i$ at period $t$ of run $r$ of the experiment, $p_{i,r}^t$ is the price set by competitors at period $t$ of run $r$ and nd $α_{i,r}$ is a firm-run fixed effect.

In [6]:
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,4.51,"""P1""",0.0
"""1_1""",1,1,2,4.0,"""P1""",-0.120003
"""1_1""",1,1,3,3.5,"""P1""",-0.133531
"""1_1""",1,1,4,3.0,"""P1""",-0.154151
"""1_1""",1,1,5,2.75,"""P1""",-0.087011
…,…,…,…,…,…,…
"""42_4""",42,4,296,1.2705,"""P2""",-0.000787
"""42_4""",42,4,297,1.269,"""P2""",-0.001181
"""42_4""",42,4,298,1.267,"""P2""",-0.001577


In [7]:
df_fe = (
    df.pivot(
        values="price_log_diff",
        index=["run_id", "period", "prompt_prefix"],
        on="firm_id",
    )
    .rename(
        {
            "1": "1_log_diff",
            "2": "2_log_diff",
            "3": "3_log_diff",
            "4": "4_log_diff",
        }
    )
    .with_columns(
        [
            pl.col("1_log_diff").shift(1).alias("1_log_diff_lag"),
            pl.col("2_log_diff").shift(1).alias("2_log_diff_lag"),
            pl.col("3_log_diff").shift(1).alias("3_log_diff_lag"),
            pl.col("4_log_diff").shift(1).alias("4_log_diff_lag"),
        ]
    )
    .filter(pl.col("period") < 100)
    # # Keep only disjoint periods
    .filter(pl.col("period") % 2 == 0)  # Adjust based on how you define disjointness
    # Alternate firms: period % 3 == 1 → firm 1, period % 3 == 2 → firm 2, period % 3 == 0 → firm 3
    .with_columns(
        [
            # Alternate firms by period % 4: 1 → firm 1, 2 → firm 2, 3 → firm 3, else firm 4
            pl.when(pl.col("period") % 4 == 1)
            .then(pl.col("1_log_diff"))
            .when(pl.col("period") % 4 == 2)
            .then(pl.col("2_log_diff"))
            .when(pl.col("period") % 4 == 3)
            .then(pl.col("3_log_diff"))
            .otherwise(pl.col("4_log_diff"))
            .alias("price_log_diff"),
            pl.when(pl.col("period") % 4 == 1)
            .then(pl.col("1_log_diff_lag"))
            .when(pl.col("period") % 4 == 2)
            .then(pl.col("2_log_diff_lag"))
            .when(pl.col("period") % 4 == 3)
            .then(pl.col("3_log_diff_lag"))
            .otherwise(pl.col("4_log_diff_lag"))
            .alias("price_log_diff_lag_own"),
            # Competitor lag average — average the other three lag columns for each case
            pl.when(pl.col("period") % 4 == 1)
            .then(
                pl.concat_list(
                    [
                        pl.col("2_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .when(pl.col("period") % 4 == 2)
            .then(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .when(pl.col("period") % 4 == 3)
            .then(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("2_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .otherwise(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("2_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .alias("price_log_diff_lag_comp_avg"),
        ]
    )
    .sort(["run_id", "period", "prompt_prefix"])
)

df_fe

run_id,period,prompt_prefix,1_log_diff,2_log_diff,3_log_diff,4_log_diff,1_log_diff_lag,2_log_diff_lag,3_log_diff_lag,4_log_diff_lag,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2,"""P1""",-0.120003,0.223144,-0.405465,0.510826,0.0,0.0,0.0,0.0,0.223144,0.0,0.0
1,4,"""P1""",-0.154151,-0.087011,0.0,0.154151,-0.133531,0.182322,-0.087011,0.182322,0.154151,0.182322,-0.01274
1,6,"""P1""",-0.09531,-0.019418,-0.035718,0.105361,-0.087011,-0.056089,0.035718,-0.441833,-0.019418,-0.056089,-0.164375
1,8,"""P1""",-0.117783,0.039221,0.0,0.04879,-0.105361,-0.019803,0.0,-0.223144,0.04879,-0.223144,-0.041721
1,10,"""P1""",-0.054067,0.038466,0.036368,0.022473,-0.051293,-0.019418,-0.018349,0.04652,0.038466,-0.019418,-0.007707
…,…,…,…,…,…,…,…,…,…,…,…,…,…
42,90,"""P2""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,92,"""P2""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,94,"""P2""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_fe = df_fe.select(
    [
        "run_id",
        "period",
        "prompt_prefix",
        "price_log_diff",
        "price_log_diff_lag_own",
        "price_log_diff_lag_comp_avg",
        # "price_log_diff_lag_comp_1",
        # "price_log_diff_lag_comp_2",
    ]
)
df_fe

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",0.223144,0.0,0.0
1,4,"""P1""",0.154151,0.182322,-0.01274
1,6,"""P1""",-0.019418,-0.056089,-0.164375
1,8,"""P1""",0.04879,-0.223144,-0.041721
1,10,"""P1""",0.038466,-0.019418,-0.007707
…,…,…,…,…,…
42,90,"""P2""",0.0,0.0,0.0
42,92,"""P2""",0.0,0.0,0.0
42,94,"""P2""",0.0,0.0,0.0


In [9]:
df_fe["prompt_prefix"].value_counts()

prompt_prefix,count
str,u32
"""P1""",1029
"""P2""",1029


## P1vsP1

In [10]:
df_fe_p1 = df_fe.filter(pl.col("prompt_prefix") == "P1").sort(["run_id", "period"])
df_fe_p1

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",0.223144,0.0,0.0
1,4,"""P1""",0.154151,0.182322,-0.01274
1,6,"""P1""",-0.019418,-0.056089,-0.164375
1,8,"""P1""",0.04879,-0.223144,-0.041721
1,10,"""P1""",0.038466,-0.019418,-0.007707
…,…,…,…,…,…
31,90,"""P1""",0.0,0.000556,0.00009
31,92,"""P1""",0.0,0.0,-0.000023
31,94,"""P1""",-0.000559,-0.006126,0.000216


In [11]:
df_fe_p1 = df_fe_p1.to_pandas()
df_fe_p1.set_index(["run_id", "period"], inplace=True)
df_fe_p1.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
1      2                 P1        0.223144                0.000000   
       4                 P1        0.154151                0.182322   
       6                 P1       -0.019418               -0.056089   
       8                 P1        0.048790               -0.223144   
       10                P1        0.038466               -0.019418   

               price_log_diff_lag_comp_avg  
run_id period                               
1      2                          0.000000  
       4                         -0.012740  
       6                         -0.164375  
       8                         -0.041721  
       10                        -0.007707

In [12]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p1,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0421
Estimator:                   PanelOLS   R-squared (Between):              0.2326
No. Observations:                1029   R-squared (Within):               0.0421
Date:                Wed, Jul 02 2025   R-squared (Overall):              0.0472
Time:                        18:37:18   Log-likelihood                    1483.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      22.094
Entities:                          21   P-value                           0.0000
Avg Obs:                       49.000   Distribution:                  F(2,1006)
Min Obs:                       49.000                                           
Max Obs:                       49.000   F-statistic (robust):             4.0588
                            

## P2vsP2

In [13]:
df_fe_p2 = df_fe.filter(pl.col("prompt_prefix") == "P2").sort(["run_id", "period"])
df_fe_p2

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
16,2,"""P2""",-0.510826,0.0,0.0
16,4,"""P2""",-0.14842,-0.139262,-0.051384
16,6,"""P2""",0.033902,-0.09844,-0.118814
16,8,"""P2""",0.030772,0.031749,-0.021842
16,10,"""P2""",0.036368,-0.105361,0.030869
…,…,…,…,…,…
42,90,"""P2""",0.0,0.0,0.0
42,92,"""P2""",0.0,0.0,0.0
42,94,"""P2""",0.0,0.0,0.0


In [14]:
df_fe_p2 = df_fe_p2.to_pandas()
df_fe_p2.set_index(["run_id", "period"], inplace=True)
df_fe_p2.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
16     2                 P2       -0.510826                0.000000   
       4                 P2       -0.148420               -0.139262   
       6                 P2        0.033902               -0.098440   
       8                 P2        0.030772                0.031749   
       10                P2        0.036368               -0.105361   

               price_log_diff_lag_comp_avg  
run_id period                               
16     2                          0.000000  
       4                         -0.051384  
       6                         -0.118814  
       8                         -0.021842  
       10                         0.030869

In [15]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p2,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0125
Estimator:                   PanelOLS   R-squared (Between):              0.0128
No. Observations:                1029   R-squared (Within):               0.0125
Date:                Wed, Jul 02 2025   R-squared (Overall):              0.0125
Time:                        18:37:18   Log-likelihood                    1130.5
Cov. Estimator:                Robust                                           
                                        F-statistic:                      6.3721
Entities:                          21   P-value                           0.0018
Avg Obs:                       49.000   Distribution:                  F(2,1006)
Min Obs:                       49.000                                           
Max Obs:                       49.000   F-statistic (robust):             1.2666
                            